In [ ]:
# pip install gradio speechrecognition gtts os dotenv openai

import gradio as gr
import speech_recognition as sr
from gtts import gTTS
import os
from dotenv import load_dotenv
from openai import OpenAI
import tempfile
import urllib

In [ ]:
# 환경 변수 로드
load_dotenv()
client = OpenAI()
api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
history = []
# OpenAI API로부터 응답 받는 함수
def get_response_from_openai(text):
    global history
    history.append({"role": "user", "content": text})
    
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=history,
    )
    history.append({"role":"assistant","content":response.choices[0].message.content})
    return response.choices[0].message.content, history

In [ ]:
# DALL-E API를 호출하여 이미지를 생성하는 함수
def get_image_from_dalle(prompt):
    response = client.images.generate(
        model="dall-e-3",
        prompt=prompt, size="1024x1024", quality = "standard", n=1
    )
    
    image_url = response.data[0].url
    urllib.request.urlretrieve(image_url, "generated_image.jpg")
    return "generated_image.jpg"

In [ ]:
# 음성 인식 및 음성 응답 생성
def process_voice(file, reset_history=False, generate_image=False):
    if reset_history:
        global history
        history=[]

    r = sr.Recognizer()
    with sr.AudioFile(file) as source:
        audio = r.record(source)
    
    try:
        # 음성 인식
        text = r.recognize_google(audio, language='ko-KR')
        # OpenAI API로 응답 받기
        response, history_display = get_response_from_openai(text)
        # 응답을 음성으로 변환
        tts = gTTS(text=response, lang='ko')
        
        # 음성을 임시 파일로 저장
        temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.mp3')
        tts.save(temp_file.name)
        temp_file.close()  # 파일 사용이 끝난 후 파일을 닫아야 함

        # 이미지 생성 옵션이 선택된 경우
        image_path = None
        if generate_image:
            image_path = get_image_from_dalle(text)
        
        return temp_file.name, history_display, image_path
    except sr.UnknownValueError:
        return "목소리를 인식하지 못했습니다.", history, None
    except sr.RequestError as e:
        return f"Speech Recognition으로부터 결과를 받을 수 없습니다. {e}", history, None

In [ ]:
# Gradio 인터페이스 설정
demo = gr.Interface(
    fn=process_voice,
    inputs=[
        gr.Audio(type="filepath", label="질문"),
        gr.Checkbox(label="대화 내역 지우기"),
        gr.Checkbox(label="이미지 생성")
    ],
    outputs=[
        gr.Audio(label="AI비서"),
        gr.JSON(label="대화 내역"),
        gr.Image(label="생성된 이미지", type="filepath")
    ],
    title="음성 AI비서",
    description="음성으로 AI와 대화를 나누어보세요. 이미지를 생성하려면 옵션을 선택하세요."
)

# 인터페이스 실행
demo.launch()